<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-PropertySets-Efficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🎓 PropertySets effizient extrahieren - ohne Element-Iteration

## Frage:
*"Wie kann ich eine Liste aller einzigartigen PropertySets und Properties aus einem IFC-Modell erzeugen, idealerweise **ohne Element-für-Element** zu iterieren."*

## Mögliche Lösung:
**Direkt `IfcPropertySet`-Instanzen abfragen statt über Elemente**

- ❌ Langsam: Element → IsDefinedBy → PropertySet
- ✅ Schnell: Direkt `model.by_type("IfcPropertySet")` → HasProperties


In [1]:
%pip install ifcopenshell


Note: you may need to restart the kernel to use updated packages.


In [2]:
import ifcopenshell
from collections import defaultdict
import time


In [3]:
# Modell laden
try:
    from google.colab import files
    print("🔼 Colab: IFC-Datei hochladen...")
    uploaded = files.upload()
    IFC_FILE = list(uploaded.keys())[0]
except:
    IFC_FILE = "Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc"
    print(f"💻 Lokal: {IFC_FILE}")

model = ifcopenshell.open(IFC_FILE)
print(f"✅ {model.schema} | {len(model.by_type('IfcElement'))} Elemente")


💻 Lokal: Modelle/BFH-25/02_BIMcollab_Example_ARC.ifc


✅ IFC2X3 | 1354 Elemente


## ✅ Die Lösung: Direkte PropertySet-Abfrage


In [4]:
# ✅ EFFIZIENT: Direkt PropertySets abfragen
start = time.time()

pset_to_props = defaultdict(set)

for pset in model.by_type("IfcPropertySet"):
    pset_name = pset.Name
    if hasattr(pset, 'HasProperties') and pset.HasProperties:
        for prop in pset.HasProperties:
            pset_to_props[pset_name].add(prop.Name)

elapsed_fast = time.time() - start

print(f"✅ Direkte Methode: {elapsed_fast:.3f}s")
print(f"✅ {len(pset_to_props)} PropertySets")
print(f"✅ {sum(len(p) for p in pset_to_props.values())} Properties total")
print(f"\n📋 Beispiele:")
for name, props in list(pset_to_props.items())[:3]:
    print(f"  • {name}: {list(props)[:3]}...")


✅ Direkte Methode: 0.052s
✅ 20 PropertySets
✅ 71 Properties total

📋 Beispiele:
  • Pset_ProjectCommon: ['BuildingPermitId']...
  • Pset_SiteCommon: ['SiteFireSafetyLevel', 'SiteSafetyClassification', 'BuildingHeightLimit']...
  • Pset_BuildingStoreyCommon: ['EntranceLevel', 'AboveGround']...


## 🐌 Vergleich: Langsame Methode (über Elemente)


In [5]:
# 🐌 LANGSAM: Über Elemente iterieren
start = time.time()

pset_to_props_slow = defaultdict(set)

for element in model.by_type("IfcObject"):
    if hasattr(element, 'IsDefinedBy') and element.IsDefinedBy:
        for rel in element.IsDefinedBy:
            if rel.is_a('IfcRelDefinesByProperties'):
                prop_def = rel.RelatingPropertyDefinition
                if prop_def and prop_def.is_a('IfcPropertySet'):
                    if hasattr(prop_def, 'HasProperties') and prop_def.HasProperties:
                        for prop in prop_def.HasProperties:
                            pset_to_props_slow[prop_def.Name].add(prop.Name)

elapsed_slow = time.time() - start

print(f"🐌 Langsame Methode: {elapsed_slow:.3f}s")
print(f"✅ {len(pset_to_props_slow)} PropertySets")
print(f"\n⚡ Speedup: {elapsed_slow/elapsed_fast:.1f}x schneller!")


🐌 Langsame Methode: 0.103s
✅ 20 PropertySets

⚡ Speedup: 2.0x schneller!


## 🎯 Als wiederverwendbare Funktion


In [6]:
def get_all_psets_and_properties(model):
    """
    Extrahiert alle PropertySets und Properties OHNE über Elemente zu iterieren.
    
    Returns:
        dict: {PropertySet-Name: set(Property-Namen)}
    """
    result = defaultdict(set)
    
    # PropertySets
    for pset in model.by_type("IfcPropertySet"):
        if hasattr(pset, 'HasProperties') and pset.HasProperties:
            for prop in pset.HasProperties:
                result[pset.Name].add(prop.Name)
    
    # Auch Quantities
    for qto in model.by_type("IfcElementQuantity"):
        if hasattr(qto, 'Quantities') and qto.Quantities:
            for q in qto.Quantities:
                result[qto.Name].add(q.Name)
    
    return dict(result)

# Testen
result = get_all_psets_and_properties(model)
print(f"✅ {len(result)} PropertySets/Quantities")
print(f"\n📋 Alle gefunden:")
for name in sorted(result.keys()):
    print(f"  • {name} ({len(result[name])} props)")


✅ 21 PropertySets/Quantities

📋 Alle gefunden:
  • BaseQuantities (35 props)
  • Material (14 props)
  • Pset_BeamCommon (4 props)
  • Pset_BuildingCommon (6 props)
  • Pset_BuildingStoreyCommon (2 props)
  • Pset_ColumnCommon (4 props)
  • Pset_CoveringCommon (2 props)
  • Pset_CurtainWallCommon (5 props)
  • Pset_DoorCommon (6 props)
  • Pset_DoorWindowGlazingType (5 props)
  • Pset_FireRatingProperties (1 props)
  • Pset_ManufacturerOccurrence (1 props)
  • Pset_ManufacturerTypeInformation (2 props)
  • Pset_ProjectCommon (1 props)
  • Pset_QuantityTakeOff (2 props)
  • Pset_RailingCommon (1 props)
  • Pset_SiteCommon (3 props)
  • Pset_SlabCommon (3 props)
  • Pset_SpaceOccupancyRequirements (1 props)
  • Pset_WallCommon (3 props)
  • Pset_WindowCommon (5 props)
